In [2]:
!python -V

Python 3.7.3


# Google スニペットを入手しよう

参考： https://qiita.com/zak_y/items/42ca0f1ea14f7046108c#comments

In [1]:
# 関数定義
import os
import datetime
import json

from time import sleep
from googleapiclient.discovery import build

GOOGLE_API_KEY          = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
CUSTOM_SEARCH_ENGINE_ID = "000000000000000000000:xxxxxxxxxxx"

DATA_DIR = 'data'

def makeDir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def get_data(qid, query, answer, dnum=10):# dnum:データ数.1page目の上から取ってくる数を指定． この引数を持つ関数にしたい．
    today = datetime.datetime.today().strftime("%Y%m%d")
    timestamp = datetime.datetime.today().strftime("%Y/%m/%d %H:%M:%S")

    makeDir(DATA_DIR)# 関数の中から外の変数は見れる

    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY) # このインスタンスが， どんなことできるのか知りたい

    page_limit = 1# どうやら，一度に10件までしかデータを取得できないみたいなので，forを回す必要がある．
    start_index = 1# データを取得し始めるindex. 1なので，頭から．
    response = []
    for n_page in range(0, page_limit):
        try:# 例外を捉える部分．Errorが無ければそのまま実行．あれば exceptへ．
            sleep(1)# 1秒待つ．何故必要かは不明．
            response.append(service.cse().list(# responseにデータを格納.
                q=query,
                cx=CUSTOM_SEARCH_ENGINE_ID,
                lr='lang_ja',
                num=dnum,# データ数. [1,10].
                start=start_index
            ).execute())
            start_index = response[n_page].get("queries").get("nextPage")[0].get("startIndex")
        except Exception as e:
            print(e)
            break
    
    # レスポンスをjson形式で保存
    save_response_dir = os.path.join(DATA_DIR, 'response')
    makeDir(save_response_dir)
    out = {'snapshot_ymd': today, 'snapshot_timestamp': timestamp, 'answer': answer, 'response': []}
    out['response'] = response
    jsonstr = json.dumps(out, ensure_ascii=False)
    with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='w') as response_file:
        response_file.write(jsonstr)
        
def make_data(qid):
    #today = datetime.datetime.today().strftime("%Y%m%d")

    response_filename = os.path.join(DATA_DIR, 'response', 'qid_' + str(qid) + '.json')
    with open(response_filename, 'r') as response_file:
        response_json = response_file.read()
        response_tmp = json.loads(response_json)
        ymd = response_tmp['snapshot_ymd']
        response = response_tmp['response']
    
    results = []
    cnt = 0
    for one_res in range(len(response)):# len(*)は,qidにつき1.あんまり意味ない．qidにつき11件以上データ取ったらそうでもない？
        if 'items' in response[one_res] and len(response[one_res]['items']) > 0:
            #print(response[one_res]['queries']['request'][0]['searchTerms'])
            answer = response_tmp['answer']# 僕の書き方依存
            question  = response[one_res]['queries']['request'][0]['searchTerms']
            results.append({'qid': qid, 'ymd': ymd, 'question': question, 'answer': answer, 'documents': []})#'title': title, 'link': link, 'snippet': snippet}) 
            for i in range(len(response[one_res]['items'])):
                cnt += 1
                display_link = response[one_res]['items'][i]['displayLink']
                title        = response[one_res]['items'][i]['title']
                link         = response[one_res]['items'][i]['link']
                snippet      = response[one_res]['items'][i]['snippet'].replace('\n', '')
                #print(results[one_res]['documents'])
                results[one_res]['documents'].append({'no': cnt, 'display_link': display_link, 'link': link, 'title': title, 'text': snippet})
                #results.append({'no': cnt, 'ymd': ymd, 'display_link': display_link, 'link': link, 'documents': [{'title': tilte, 'text': snippet}]})#'title': title, 'link': link, 'snippet': snippet}) 
    save_results_dir = os.path.join(DATA_DIR, 'results')
    makeDir(save_results_dir)
    
    # レスポンスをjson形式で保存
    jsonstr = json.dumps(results, ensure_ascii=False)
    with open(os.path.join(save_results_dir, 'qid_' + str(qid) + '.json'), mode='w') as results_file:
        results_file.write(jsonstr)

## (q,a)取得

In [13]:
import gzip
import json

dataset = []
with gzip.open("dataset/all-v1.0.json.gz", "rt", encoding="utf-8") as fp:
    for line in fp:
        data = json.loads(line)
        if data["documents"]:
            dataset.append(data)

## Googleスニペットを入手
3月-11月は16時がリセットタイミングらしい． 1query 10 件データ取ってくるなら， 100query/day. 

In [3]:
start = 400
dataset[start]['qid']

428

In [4]:
hoge# うっかり実行しないようにhoge
for i in range(start, start + 100):
    qid, q, a = dataset[i]['qid'], dataset[i]['question'], dataset[i]['answer']
    get_data(qid, q, a)

'NoneType' object is not subscriptable


In [39]:
# 取得したデータを確認
def view_data(qid):
    dataset = []
    save_response_dir = os.path.join(DATA_DIR, 'response')
    with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='r') as fp:
        for line in fp:
            data = json.loads(line)
            dataset.append(data)
        return dataset

## 加工整形処理

qid が連続じゃないので，元データのファイルの読み込みに，少し工夫が必要になった

In [3]:
start = 10
dnum = 90
dataset[start]['qid']

12

In [4]:
hoge #うっかり実行しないようにhgoe
for i in range(start, start + dnum):
    make_data(dataset[i]['qid'])

## 取得したデータを確認

In [14]:
import os
import json
DATA_DIR = 'data'

save_response_dir = os.path.join(DATA_DIR, 'results')
results = []
for i in range(10):
    qid = dataset[i]['qid']
    with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='r') as fp:
        for line in fp:
            result = json.loads(line)
            results.append(result)

データを見てみて
- snippet の末には \xa0... がある．
- 自分で Google 検索を行った結果と少し異なる．サイトの表示順が違った．APIの検索方法とかは調べた方が良さそう．
- スニペットの選択方法が気になる．
- クエリに対して，何件スニペットを取ってくるかは未定(今回なら10件だけ取ってきた．取得したコードのページ1とした部分.1ページに表示されるサイト数は10件)

In [5]:
results[0][0].keys() # results[1][2]. 1:qid, 2:qidに対応するデータ.10件以上とると，index:0以外も使う．

dict_keys(['qid', 'ymd', 'question', 'answer', 'documents'])

In [6]:
a =results[0][0]['documents']
len(a)

10

In [7]:
a[0] 

{'no': 1,
 'display_link': 'qss.quiz-island.site',
 'link': 'http://qss.quiz-island.site/abcgo-info/1',
 'title': 'abc / EQIDEN 過去問検索サイト『 abc / EQIDEN Search (β)』',
 'text': '大会名, abc the first (2003). 種別, ペーパー筆記. 問題, 「abc ～the first～」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう？ 正解, 26文字. 解説. 戻る. ↑'}

In [15]:
# データ確認
results[0]

[{'qid': 1,
  'ymd': '20200825',
  'question': '「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?',
  'answer': '26文字',
  'documents': [{'no': 1,
    'display_link': 'qss.quiz-island.site',
    'link': 'http://qss.quiz-island.site/abcgo-info/1',
    'title': 'abc / EQIDEN 過去問検索サイト『 abc / EQIDEN Search (β)』',
    'text': '大会名, abc the first (2003). 種別, ペーパー筆記. 問題, 「abc ～the first～」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう？ 正解, 26文字. 解説. 戻る. ↑'},
   {'no': 2,
    'display_link': 'web.sfc.wide.ad.jp',
    'link': 'https://web.sfc.wide.ad.jp/~kaori/quiz.pdf',
    'title': '26文字 「abc ～the first～」へようこそ！さて、ABC・・・と始まる ...',
    'text': '1. 26文字. 「abc ～the first～」へようこそ！さて、ABC・・・と始まるアルファベットは、全部で何文字で. しょう？ 2. 骨川（滑川 ... 織田信長、豊臣秀吉、徳川家康という３人の戦国武将の性格を表現するのに用いられる鳥は何でしょ. う？ 6.'},
   {'no': 3,
    'display_link': 'quizlet.com',
    'link': 'https://quizlet.com/jp/470527690/abc1-flash-cards/',
    'title': 'abc1 Flashcards',
    'text': '「abc ～the first～」へようこそ！さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう？ 26文字. 

In [16]:
results[1]

[{'qid': 2,
  'ymd': '20200825',
  'question': '人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?',
  'answer': '骨川',
  'documents': [{'no': 1,
    'display_link': 'quiz-schedule.info',
    'link': 'https://quiz-schedule.info/quiz_no_mori/data/aquiz0050.html',
    'title': '問題データ',
    'text': '番号, 問題, 解答. 1, 「abc ～the first～」へようこそ！さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう？ 26文字. 2, 人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう？'},
   {'no': 2,
    'display_link': 'ja.wikipedia.org',
    'link': 'https://ja.wikipedia.org/wiki/%E9%AA%A8%E5%B7%9D%E3%82%B9%E3%83%8D%E5%A4%AB',
    'title': '骨川スネ夫 - Wikipedia',
    'text': '骨川 スネ夫（ほねかわ スネお）は、藤子・F・不二雄の漫画作品『ドラえもん』に登場する架空の人物。野比のび太のクラスメイト。2月生まれ（何日かは不明、午後5時5分に生まれたことがビデオに残されている）（方倉設定では3月28日\xa0...'},
   {'no': 3,
    'display_link': 'note.com',
    'link': 'https://note.com/ashi_tomo/n/nfb0c6ea2219e',
    'title': '【三題話】ドラえもんのキャラクターたちの名字は実在するのか ...',
    'text': '2019年11月29日 ... このあまりに有名なマンガ、登場人物が変わった名前なのも特徴の一つで、野比のび太とか、出木杉君とか、実際にはいなさそうな ... ところでそれこそいま調べたのです

In [17]:
results[2]

[{'qid': 3,
  'ymd': '20200825',
  'question': '格闘家ボブ・サップの出身国はどこでしょう?',
  'answer': 'アメリカ',
  'documents': [{'no': 1,
    'display_link': 'ja.wikipedia.org',
    'link': 'https://ja.wikipedia.org/wiki/%E3%83%9C%E3%83%96%E3%83%BB%E3%82%B5%E3%83%83%E3%83%97',
    'title': 'ボブ・サップ - Wikipedia',
    'text': 'ボブ・サップ（Bob Sapp、1973年9月22日 - ）は、アメリカ合衆国のキックボクサー、プロレスラー、総合格闘家、タレント、俳優、元アメリカンフットボール選手。コロラド州コロラドスプリングス出身。元はNFL選手であったものの、\xa0...'},
   {'no': 2,
    'display_link': 'twitter.com',
    'link': 'https://twitter.com/botquiz/status/100627366824525824',
    'title': 'botquiz on Twitter: "格闘家ボブ・サップの出身国はどこでしょう ...',
    'text': '2011年8月8日 ... botquiz · @botquiz. abcとクイズロードを15分毎につぶやきます。 Joined June 2011. Tweets. © 2020 Twitter; About · Help Center · Terms · Privacy policy · Cookies · Ads info. Dismiss. Close. Previous. Next. Close. Go to a\xa0...'},
   {'no': 3,
    'display_link': 'www.nlp.ecei.tohoku.ac.jp',
    'link': 'https://www.nlp.ecei.tohoku.ac.jp/projects/jaqket/',
    'title': 'JAQKET:ク

In [18]:
results[3]

[{'qid': 4,
  'ymd': '20200825',
  'question': 'ロシア語で「城」という意味がある、ロシアの大統領府の別名は何でしょう?',
  'answer': 'クレムリン',
  'documents': [{'no': 1,
    'display_link': 'ja.wikipedia.org',
    'link': 'https://ja.wikipedia.org/wiki/%E3%82%AF%E3%83%AC%E3%83%A0%E3%83%AA%E3%83%B3',
    'title': 'クレムリン - Wikipedia',
    'text': "クレムリン（露：Кремль、Kreml'）は、ロシア連邦の首都、モスクワ市の中心を流れるモスクワ川沿いにある旧ロシア帝国の宮殿。「Kremlin」は、英語やフランス語などでの表記。ソビエト連邦時代にはソ連共産党の中枢が置かれたことから、ソ連共産党の別名としても用いられた。現在もロシア連邦の大統領府や大統領官邸が置かれているため、ロシア政府の代名詞として用いられる。正面には赤の広場がある。 ロシア語では「クレムリ」となり、「城塞」を意味する。 ... された際には、この宮殿のみ鉄筋コンクリートとガラス張りという近代性ゆえに世界遺産の指定対象から外された。"},
   {'no': 2,
    'display_link': 'quiz-schedule.info',
    'link': 'https://quiz-schedule.info/quiz_no_mori/data/aquiz0050.html',
    'title': '問題データ',
    'text': '3, 格闘家ボブ・サップの出身国はどこでしょう？ アメリカ. 4, ロシア語で「城」という意味がある、ロシアの大統領府の別名は何でしょう？ クレムリン. 5, 織田信長、豊臣秀吉、徳川家康という３人の戦国武将の性格を表現するのに用いられる\xa0...'},
   {'no': 3,
    'display_link': 'quizlet.com',
    'link': 'https://quizlet.com/jp/513657403/%E3%82%AF%E3%82%

In [19]:
results[4]

[{'qid': 5,
  'ymd': '20200825',
  'question': '織田信長、豊臣秀吉、徳川家康という3人の戦国武将の性格を表現するのに用いられる鳥は何でしょう?',
  'answer': 'ホトトギス',
  'documents': [{'no': 1,
    'display_link': 'twitter.com',
    'link': 'https://twitter.com/botquiz/status/290529936454844418',
    'title': 'botquiz on Twitter: "織田信長、豊臣秀吉、徳川家康という3人の戦国 ...',
    'text': 'Embed Tweet. 織田信長、豊臣秀吉、徳川家康という３人の戦国武将の性格を表現するのに用いられる鳥は何でしょう？／【ホトトギス】. 10:45 AM - 13 Jan 2013. 0 replies 0 retweets 0 likes. Reply. Retweet. Retweeted. Like. Liked. Back to\xa0...'},
   {'no': 2,
    'display_link': 'quizlet.com',
    'link': 'https://quizlet.com/jp/470527690/abc1-flash-cards/',
    'title': 'abc1 Flashcards',
    'text': '格闘家ボブ・サップの出身国はどこでしょう？ アメリカ. ロシア語で「城」という意味がある、ロシアの大統領府の別名は何でしょう？ クレムリン. 織田信長、豊臣秀吉、徳川家康という３人の戦国武将の性格を表現するのに用いられる鳥は\xa0...'},
   {'no': 3,
    'display_link': 'matome.naver.jp',
    'link': 'https://matome.naver.jp/odai/2149430547423419401/2150652480560008203',
    'title': 'tweet : クイズ王になりたい人のための問題集【鳥】 - NAVER まとめ',
    'text': '2017